In [1]:
import os
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

In [2]:
data_path = "../utils/raw_data_processing/processed_corpus/processed_data.csv"

In [85]:
df = pd.read_csv(data_path)

In [86]:
copy_df = df.copy()
df.head()

,Unnamed: 0,in_speaker,input,out_speaker,output
0,0,Scene,A corridor at a sperm bank.,Sheldon,So if a photon is directed through a plane wit...
1,1,Leonard,"Agreed, what’s your point?",Sheldon,"There’s no point, I just think it’s a good ide..."
2,2,Receptionist,"If you have to ask, maybe you shouldn’t be here.",Sheldon,I think this is the place.
3,3,Leonard,"What, are you kidding? You’re a semi-pro.",Sheldon,No. We are committing genetic fraud. There’s n...
4,4,Leonard,"Sheldon, this was your idea. A little extra mo...",Sheldon,"I know, and I do yearn for faster downloads, b..."


In [51]:
#number of sentences in docs.
print(len(df))

9916


In [6]:
# 0. lower all the sentece before any processing
# 1. tokenize all sentences and check length of each vector and then decide the appropriate inout and output vectore size. 
# 2. for experiment 1, do least preprocessing. try with just raw encoded data. 
# 3. glove embedings for the vectors, with vector size of 100. 
# 4. vanilla BLSTM model. 
# 5. as it is a text generation task, no need to remove stop words or lemmatize, else we will loose important gramatic information

In [7]:
#Check input leghth after tokenization. 

In [55]:
df["tokenized_input"] = df["input"].apply(lambda x: word_tokenize(x.lower()) if type(x) == str else x)
df["tokenized_output"] = df["output"].apply(lambda x: word_tokenize(x.lower()) if type(x) == str else x)

In [60]:
df["input_length"] = df["tokenized_input"].apply(len)
df["output_length"] = df["tokenized_output"].apply(lambda x: len(x) if type(x) == list else 1)


In [10]:
df["input_length"].describe()

count    9916.000000
mean       12.862545
std        11.370839
min         1.000000
25%         5.000000
50%        10.000000
75%        17.000000
max       148.000000
Name: input_length, dtype: float64

In [61]:
df["output_length"].describe()

count    9916.000000
mean       19.312324
std        16.562703
min         1.000000
25%         8.000000
50%        15.000000
75%        26.000000
max       245.000000
Name: output_length, dtype: float64

In [13]:
#from above summary, 75% of data point have word count of equal or less than 17. 
# calculating summary for the 90% of the data. 

In [74]:
x = (df["input_length"] <= 25).value_counts()
print(x)

8811/len(df["input_length"])

True     8811
False    1105
Name: input_length, dtype: int64


0.8885639370713998

In [76]:
x = (df["output_length"] <= 25).value_counts()
print(x)

7411/len(df["output_length"])

True     7411
False    2505
Name: output_length, dtype: int64


0.7473779749899153

In [ ]:
# 88.85% input data have the word count less than 25, hence considering 25 as word count length.
# 74.73% output data have the word count less than 25, hence considering 25 as word count length.
# 25 --> sequence length, larger sequence length might hamper vanishing gradient problem in lstm. 

In [77]:
df.head()

,Unnamed: 0,in_speaker,input,out_speaker,output,tokenized_input,tokenized_output,input_length,output_length
0,0,Scene,A corridor at a sperm bank.,Sheldon,So if a photon is directed through a plane wit...,"[a, corridor, at, a, sperm, bank, .]","[so, if, a, photon, is, directed, through, a, ...",7,65
1,1,Leonard,"Agreed, what’s your point?",Sheldon,"There’s no point, I just think it’s a good ide...","[agreed, ,, what, ’, s, your, point, ?]","[there, ’, s, no, point, ,, i, just, think, it...",8,19
2,2,Receptionist,"If you have to ask, maybe you shouldn’t be here.",Sheldon,I think this is the place.,"[if, you, have, to, ask, ,, maybe, you, should...","[i, think, this, is, the, place, .]",14,7
3,3,Leonard,"What, are you kidding? You’re a semi-pro.",Sheldon,No. We are committing genetic fraud. There’s n...,"[what, ,, are, you, kidding, ?, you, ’, re, a,...","[no, ., we, are, committing, genetic, fraud, ....",12,43
4,4,Leonard,"Sheldon, this was your idea. A little extra mo...",Sheldon,"I know, and I do yearn for faster downloads, b...","[sheldon, ,, this, was, your, idea, ., a, litt...","[i, know, ,, and, i, do, yearn, for, faster, d...",20,58


In [78]:
#using 75%-25% split for training and testing after randomizing the data points, after randomizing datapoints

In [95]:
random_df = copy_df.sample(frac = 1)

In [101]:
train_df = random_df[["in_speaker", "input", "out_speaker","output"]][:int(len(random_df)*0.75)]
test_df = random_df[["in_speaker", "input", "out_speaker","output"]][int(len(random_df)*0.75):]

In [103]:
if not os.path.exists("../downloads/preprocessed_data"):
    os.mkdir("../downloads/preprocessed_data")

train_df.to_csv("../downloads/preprocessed_data/train.csv", index = False)
train_df.to_csv("../downloads/preprocessed_data/test.csv", index = False)

In [83]:
os.path.exists("../downloads")

True

In [87]:
random_df.shape

(9916, 9)

In [99]:
train_df.head()

,Unnamed: 0,in_speaker,input,out_speaker,output
594,16,Leonard,"Easy, easy Okay! Now we’ve got an inclined pl...",Sheldon,Exactly half.
4412,20,Leonard,Got it.,Sheldon,"Although, for the record, on one occasion, she..."
7630,0,Penny,What are you working on?,Sheldon,Can’t talk. In the zone.
930,41,Leonard,"So, we’ll just order three entrees.",Sheldon,"Fine, what do you want to eliminate, and who g..."
2072,74,Leonard,"I wonder a lot of things about you, Sheldon, b...",Sheldon,"Well, those are indicators that I’m farther al..."


In [104]:
"sdads.csv".endswith(".csv")

True